<a href="https://colab.research.google.com/github/MLNETO22/Algoritmos/blob/main/Algoritmos_Grafos_%7C_%C3%81rvore_Bin%C3%A1ria_e_AVL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
%%writefile arvore_binaria.py

# arvore_binaria.py

class Node:
    def __init__(self, key, data=None):
        self.key = key
        self.data = data
        self.left = None
        self.right = None

class ArvoreBinaria:
    """
    Implementa uma Árvore Binária de Busca (BST).
    Complexidade:
    - Inserção: O(h) (≈ O(n) no pior caso)
    - Busca: O(h) (≈ O(n) no pior caso)
    - Remoção: O(h) (≈ O(n) no pior caso)
    - Percursos: O(n)
    """
    def __init__(self):
        self.root = None

    def insert(self, key, data=None):
        if self.root is None:
            self.root = Node(key, data)
        else:
            self._insert(self.root, key, data)

    def _insert(self, node, key, data):
        if key < node.key:
            if node.left is None:
                node.left = Node(key, data)
            else:
                self._insert(node.left, key, data)
        elif key > node.key:
            if node.right is None:
                node.right = Node(key, data)
            else:
                self._insert(node.right, key, data)

    def search(self, key):
        return self._search(self.root, key)

    def _search(self, node, key):
        if node is None or node.key == key:
            return node
        if key < node.key:
            return self._search(node.left, key)
        return self._search(node.right, key)

    def remove(self, key):
        self.root = self._remove(self.root, key)

    def _remove(self, node, key):
        if node is None:
            return node
        if key < node.key:
            node.left = self._remove(node.left, key)
        elif key > node.key:
            node.right = self._remove(node.right, key)
        else:
            if node.left is None:
                return node.right
            elif node.right is None:
                return node.left
            temp = self._min_value_node(node.right)
            node.key = temp.key
            node.data = temp.data
            node.right = self._remove(node.right, temp.key)
        return node

    def _min_value_node(self, node):
        current = node
        while current.left is not None:
            current = current.left
        return current

    def inorder(self):
        result = []
        self._inorder(self.root, result)
        return result

    def _inorder(self, node, result):
        if node:
            self._inorder(node.left, result)
            result.append((node.key, node.data))
            self._inorder(node.right, result)

    def preorder(self):
        result = []
        self._preorder(self.root, result)
        return result

    def _preorder(self, node, result):
        if node:
            result.append((node.key, node.data))
            self._preorder(node.left, result)
            self._preorder(node.right, result)

    def postorder(self):
        result = []
        self._postorder(self.root, result)
        return result

    def _postorder(self, node, result):
        if node:
            self._postorder(node.left, result)
            self._postorder(node.right, result)
            result.append((node.key, node.data))

Overwriting arvore_binaria.py


In [11]:
%%writefile arvore_avl.py

# arvore_avl.py

from arvore_binaria import ArvoreBinaria, Node

class AVLNode(Node):
    def __init__(self, key, data=None):
        super().__init__(key, data)
        self.height = 1

class ArvoreAVL(ArvoreBinaria):
    """
    Extende a BST para AVL com balanceamento.
    Complexidade: O(log n) para inserção, busca e remoção.
    """
    def __init__(self):
        super().__init__()

    def _height(self, node):
        if not node:
            return 0
        return node.height

    def _balance_factor(self, node):
        if not node:
            return 0
        return self._height(node.left) - self._height(node.right)

    def _update_height(self, node):
        node.height = 1 + max(self._height(node.left), self._height(node.right))

    def _rotate_right(self, y):
        x = y.left
        T2 = x.right
        x.right = y
        y.left = T2
        self._update_height(y)
        self._update_height(x)
        return x

    def _rotate_left(self, x):
        y = x.right
        T2 = y.left
        y.left = x
        x.right = T2
        self._update_height(x)
        self._update_height(y)
        return y

    def _insert(self, node, key, data):
        if node is None:
            return AVLNode(key, data)
        if key < node.key:
            node.left = self._insert(node.left, key, data)
        elif key > node.key:
            node.right = self._insert(node.right, key, data)
        else:
            return node  # No duplicates

        self._update_height(node)
        balance = self._balance_factor(node)

        # Left Left
        if balance > 1 and key < node.left.key:
            return self._rotate_right(node)
        # Right Right
        if balance < -1 and key > node.right.key:
            return self._rotate_left(node)
        # Left Right
        if balance > 1 and key > node.left.key:
            node.left = self._rotate_left(node.left)
            return self._rotate_right(node)
        # Right Left
        if balance < -1 and key < node.right.key:
            node.right = self._rotate_right(node.right)
            return self._rotate_left(node)

        return node

    def insert(self, key, data=None):
        self.root = self._insert(self.root, key, data)

    def _remove(self, node, key):
        if not node:
            return node
        if key < node.key:
            node.left = self._remove(node.left, key)
        elif key > node.key:
            node.right = self._remove(node.right, key)
        else:
            if node.left is None:
                return node.right
            elif node.right is None:
                return node.left
            temp = self._min_value_node(node.right)
            node.key = temp.key
            node.data = temp.data
            node.right = self._remove(node.right, temp.key)

        if node is None:
            return node

        self._update_height(node)
        balance = self._balance_factor(node)

        # Left Left
        if balance > 1 and self._balance_factor(node.left) >= 0:
            return self._rotate_right(node)
        # Left Right
        if balance > 1 and self._balance_factor(node.left) < 0:
            node.left = self._rotate_left(node.left)
            return self._rotate_right(node)
        # Right Right
        if balance < -1 and self._balance_factor(node.right) <= 0:
            return self._rotate_left(node)
        # Right Left
        if balance < -1 and self._balance_factor(node.right) > 0:
            node.right = self._rotate_right(node.right)
            return self._rotate_left(node)

        return node

    def remove(self, key):
        self.root = self._remove(self.root, key)

Overwriting arvore_avl.py


In [12]:
%%writefile grafo.py

# grafo.py

import heapq
from collections import deque

class Grafo:
    """
    Implementa grafos usando lista de adjacência.
    Complexidade:
    - BFS/DFS: O(V + E)
    - Dijkstra: O(E log V) (com heap)
    """
    def __init__(self):
        self.adj = {}  # {vertex: [(neighbor, weight), ...]}

    def add_vertex(self, vertex):
        if vertex not in self.adj:
            self.adj[vertex] = []

    def add_edge(self, u, v, weight=1):
        self.add_vertex(u)
        self.add_vertex(v)
        self.adj[u].append((v, weight))
        self.adj[v].append((u, weight))  # Undirected

    def bfs(self, start):
        visited = set()
        queue = deque([start])
        result = []
        while queue:
            vertex = queue.popleft()
            if vertex not in visited:
                visited.add(vertex)
                result.append(vertex)
                for neighbor, _ in self.adj[vertex]:
                    if neighbor not in visited:
                        queue.append(neighbor)
        return result

    def dfs(self, start):
        visited = set()
        stack = [start]
        result = []
        while stack:
            vertex = stack.pop()
            if vertex not in visited:
                visited.add(vertex)
                result.append(vertex)
                for neighbor, _ in self.adj[vertex]:
                    if neighbor not in visited:
                        stack.append(neighbor)
        return result

    def dijkstra(self, start, end):
        pq = [(0, start)]  # (distance, vertex)
        distances = {vertex: float('inf') for vertex in self.adj}
        distances[start] = 0
        previous = {vertex: None for vertex in self.adj}
        while pq:
            current_dist, current = heapq.heappop(pq)
            if current == end:
                break
            if current_dist > distances[current]:
                continue
            for neighbor, weight in self.adj[current]:
                distance = current_dist + weight
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    previous[neighbor] = current
                    heapq.heappush(pq, (distance, neighbor))
        if distances[end] == float('inf'):
            return None, None
        # Rebuild path
        path = []
        current = end
        while current is not None:
            path.append(current)
            current = previous[current]
        path.reverse()
        return path, distances[end]

Overwriting grafo.py


In [13]:
%%writefile main.py

# main.py

from arvore_avl import ArvoreAVL
from grafo import Grafo

def print_complexities():
    print("\nAnálises de Complexidade:")
    print("Operação\tEstrutura\tComplexidade")
    print("Inserção\tÁrvore Binária\tO(h) (≈ O(n) pior caso)")
    print("Inserção\tAVL\t\tO(log n)")
    print("Busca em largura (BFS)\tGrafo\t\tO(V + E)")
    print("Caminho mínimo (Dijkstra)\tGrafo ponderado\tO(E log V)")

def main():
    avl = ArvoreAVL()
    while True:
        print("\nMenu:")
        print("1. Cadastrar cidade")
        print("2. Remover cidade")
        print("3. Visualizar percursos (pré-ordem, em-ordem, pós-ordem)")
        print("4. Gerenciar grafo de bairros de uma cidade")
        print("5. Exibir complexidades")
        print("6. Sair")
        op = input("Escolha uma opção: ")
        if op == '1':
            name = input("Nome da cidade: ")
            city_id = input("ID da cidade: ")
            data = {'id': city_id, 'graph': Grafo()}
            avl.insert(name, data)
            print(f"Cidade {name} cadastrada.")
        elif op == '2':
            name = input("Nome da cidade a remover: ")
            avl.remove(name)
            print(f"Cidade {name} removida.")
        elif op == '3':
            print("Pré-ordem:", [key for key, _ in avl.preorder()])
            print("Em-ordem:", [key for key, _ in avl.inorder()])
            print("Pós-ordem:", [key for key, _ in avl.postorder()])
        elif op == '4':
            name = input("Nome da cidade: ")
            node = avl.search(name)
            if node:
                graph = node.data['graph']
                while True:
                    print("\nSubmenu Grafo:")
                    print("a. Adicionar bairro")
                    print("b. Adicionar conexão viária")
                    print("c. Executar BFS")
                    print("d. Executar DFS")
                    print("e. Calcular caminho mínimo (Dijkstra)")
                    print("f. Voltar")
                    sub_op = input("Escolha: ")
                    if sub_op == 'a':
                        bairro = input("Nome do bairro: ")
                        graph.add_vertex(bairro)
                    elif sub_op == 'b':
                        u = input("Bairro origem: ")
                        v = input("Bairro destino: ")
                        weight = float(input("Distância/peso: "))
                        graph.add_edge(u, v, weight)
                    elif sub_op == 'c':
                        start = input("Bairro inicial: ")
                        print("BFS:", graph.bfs(start))
                    elif sub_op == 'd':
                        start = input("Bairro inicial: ")
                        print("DFS:", graph.dfs(start))
                    elif sub_op == 'e':
                        start = input("Bairro inicial: ")
                        end = input("Bairro final: ")
                        path, dist = graph.dijkstra(start, end)
                        if path:
                            print(f"Caminho: {path}, Distância: {dist}")
                        else:
                            print("Sem caminho.")
                    elif sub_op == 'f':
                        break
            else:
                print("Cidade não encontrada.")
        elif op == '5':
            print_complexities()
        elif op == '6':
            break

if __name__ == "__main__":
    main()

Overwriting main.py


In [ ]:
!python main.py


Menu:
1. Cadastrar cidade
2. Remover cidade
3. Visualizar percursos (pré-ordem, em-ordem, pós-ordem)
4. Gerenciar grafo de bairros de uma cidade
5. Exibir complexidades
6. Sair
Escolha uma opção: 